**author**: lukethompson@gmail.com<br>
**date**: 16 Nov 2016<br>
**language**: Python 3.5<br>
**conda enviroment**: emp-py3<br>
**license**: unlicensed<br>

## otu_trading_cards.ipynb

Generate LaTeX macros and plots for a 'trading card' for any given Deblur OTU.

In [ ]:
import pandas as pd
import numpy as np
import biom
import wikipedia
import re
import os
import errno
import math
import matplotlib.pyplot as plt
import seaborn as sns
import empcolors
import GetV4Region
%matplotlib inline

In [ ]:
# seaborn plot settings
sns.set_context('talk')
sns.set(style='white', palette='muted', color_codes=True)

In [ ]:
# input
path_map = '~/emp/metadata-refine/emp_qiime_mapping_subset_2k.tsv'
path_otus = '~/emp/analyses-otus/otu_summary.emp_deblur_100bp.subset_2k.rare_5000.tsv'
path_rdp = '~/emp/analyses-otus/otu_seqs_top_500_prev.emp_deblur_100bp.subset_2k.rare_5000.tsv'
obs_column = 'observations_deblur_100bp'
trim_length = 100
num_samples = 1856 # 2000 for 90bp, 1856 for 100bp, 975 for 150bp
rarefaction_depth = 5000
subset = '2k'

# output
path_output = '~/emp/analyses-otus'

In [ ]:
# dataframe of colors
df_colors = pd.DataFrame.from_dict(empcolors.get_empo_cat_color(returndict=True), orient='index')
df_colors.sort_index(inplace=True)

In [ ]:
# get wikipedia entry for genus or higher (lowest taxonomic level that begins (position 3) with capital letter)
def get_wikipedia(my_taxonomy):
    for level in reversed(my_taxonomy.split('; ')):
        if len(level) > 3:
            if level[3].isupper():
                title = level[3:]
                print(title)
                try:
                    entry = wikipedia.page(title)
                    return('%s\t%s' % (title, entry.summary))
                except wikipedia.exceptions.DisambiguationError as e:
                    return('%s\t%s has multiple options: %s' % (title, title, e.options))
                except wikipedia.exceptions.PageError as e:
                    return('%s\t%s has no Wikipedia page.' % (title, title))
                break

In [ ]:
# make directory if doesn't already exist
def make_directory(path):
    try:
        os.mkdir(path)
    except OSError as exc:
        if exc.errno != errno.EEXIST:
            raise exc
        pass

In [ ]:
# read mapping file
df_map = pd.read_csv(path_map, sep='\t', index_col=0)
# read otu summary
df_otus = pd.read_csv(path_otus, sep='\t', index_col=0)
# read rdp taxonomy (index = sequence)
df_rdp = pd.read_csv(path_rdp, sep='\t', index_col=0)

### Option 1: Get OTUs matching to query 16S

In [ ]:
for path in ['/Users/luke.thompson/moi/RefSeq_16S/Bacteroides_coprocola_M16.NR_041278.fasta',
             '/Users/luke.thompson/moi/RefSeq_16S/Bacteroides_dorei_175.NR_041351.fasta',
             '/Users/luke.thompson/moi/RefSeq_16S/Bacteroides_intestinalis_341.NR_041307.fasta']:
    GetV4Region.GetV4(inputname=path, 
                  fprimer='GTGCCAGC[AC]GCCGCGGTAA',
                  rprimer='ATTAGA[AT]ACCC[CGT][AGT]GTAGTCC',
                  length=100,
                  remove_ambig=False,
                  keep_primers=False,
                  skip_reverse=False)

In [ ]:
bacteroides = [
        'TACGGAGGATGCGAGCGTTATCCGGATTTATTGGGTTTAAAGGGAGCGCAGACGGGAGATTAAGTCAGTTGTGAAAGTTTGCGGCTCAACCGTAAAATTG',
        'TACGGAGGATCCGAGCGTTATCCGGATTTATTGGGTTTAAAGGGAGCGTAGATGGATGTTTAAGTCAGTTGTGAAAGTTTGCGGCTCAACCGTAAAATTG',
        'TACGGAGGATCCGAGCGTTATCCGGATTTATTGGGTTTAAAGGGAGCGTAGGCGGATTATTAAGTCAGTTGTGAAAGTTTGCGGCTCAACCGTAAAATTG',
    ]

In [ ]:
set(bacteroides)

In [ ]:
df_otus_top = df_otus[df_otus['sequence'].isin(bacteroides)]

##### TO DO: Add fasta name to df_otus_top so we can track the strains

### Option 2: Get top OTUs by num_samples OR total_obs

In [ ]:
#df_otus_top = df_otus.sort_values('num_samples', ascending=False).head(10)
df_otus_top = df_otus.sort_values('total_obs', ascending=False).head(10)

In [ ]:
df_otus_top

In [ ]:
# add wikipedia summary
df_otus_top['wikipedia'] = df_otus_top['taxonomy'].apply(get_wikipedia)
df_otus_top['title'] = [value.split('\t')[0] for value in df_otus_top['wikipedia']]
df_otus_top['wikipedia'] = [value.split('\t')[1] for value in df_otus_top['wikipedia']]

In [ ]:
for index, row in df_otus_top.iterrows():
    
    # store the relevant values
    sequence = row['sequence']
    taxonomy_gg = re.sub(r'_', r'\_', row['taxonomy'])
    try:
        df_rdp.loc[row['sequence']]
        taxonomy_rdp = re.sub(r'_', r'\_', df_rdp.loc[row['sequence']]['lineage_count'])
        species_1st_rdp = df_rdp.loc[row['sequence']]['species_1st_count']
        species_2nd_rdp = df_rdp.loc[row['sequence']]['species_2nd_count']
        species_3rd_rdp = df_rdp.loc[row['sequence']]['species_3rd_count']
    except:
        taxonomy_rdp = ''
        species_1st_rdp = ''
        species_2nd_rdp = ''
        species_3rd_rdp = ''
    wikipedia = row['wikipedia']
    wikipedia = re.sub(r'"', r'``', wikipedia)
    wikipedia = re.sub(u'\u201D', r"''", wikipedia) # need to replace unicode backward double quote
    title = row['title']
    prevalencePercent = row['num_samples_frac'] * 100
    prevalenceRank = str(row['num_samples_rank'] + 1)
    abundancePercent = row['total_obs_frac'] * 100
    abundanceRank = str(row['total_obs_rank'] + 1)
    numOTUs = str(df_otus.shape[0])
    trimLength = str(trim_length)
    numSamples = str(num_samples)
    rarefactionDepth = str(rarefaction_depth)

    # MAKE DIRECTORY
    make_directory('%s/card_%sbp_subset%s_rare%s_rank%s_%s' % (path_output, 
            trimLength, subset, rarefactionDepth, prevalenceRank, title))
    
    # CREATE MACROS FILE
    with open('%s/card_%sbp_subset%s_rare%s_rank%s_%s/macros.tex' % (path_output, 
            trimLength, subset, rarefactionDepth, prevalenceRank, title), 'w') as target:
        # SEQUENCE
        target.write(r'\def\sequence{')
        # first 50bp
        target.write(sequence[:50])
        # next 50bp
        target.write('\n')
        target.write(sequence[50:100])
        # next 50bp if > 100bp
        if len(sequence) > 100:
            target.write('\n')
            target.write(sequence[100:150])
        target.write('}\n')
        # TAXONOMY
        target.write(r'\def\taxonomyGG{')
        target.write(taxonomy_gg)
        target.write('}\n')
        target.write(r'\def\taxonomyRDP{')
        target.write(taxonomy_rdp)
        target.write('}\n')
        target.write(r'\def\speciesA{')
        target.write(species_1st_rdp)
        target.write('}\n')
        target.write(r'\def\speciesB{')
        target.write(species_2nd_rdp)
        target.write('}\n')
        target.write(r'\def\speciesC{')
        target.write(species_3rd_rdp)
        target.write('}\n')
        # WIKIPEDIA
        target.write(r'\def\wikipedia{')
        if len(wikipedia) > 650:
            target.write(wikipedia[:650])
            target.write('...')
        else:
            target.write(wikipedia)
        target.write('}\n')
        # PREVALENCE
        target.write(r'\def\prevalencePercent{')
        target.write('{:0.2f}'.format(prevalencePercent))
        target.write('}\n')
        target.write(r'\def\prevalenceRank{')
        target.write(prevalenceRank)
        target.write('}\n')
        # ABUNDANCE
        target.write(r'\def\abundancePercent{')
        target.write('{:0.3f}'.format(abundancePercent))
        target.write('}\n')
        target.write(r'\def\abundanceRank{')
        target.write(abundanceRank)
        target.write('}\n')
        # METHODS/MISC
        target.write(r'\def\numOTUs{')
        target.write(numOTUs)
        target.write('}\n')
        target.write(r'\def\trimLength{')
        target.write(trimLength)
        target.write('}\n')
        target.write(r'\def\numSamples{')
        target.write(numSamples)
        target.write('}\n')
        target.write(r'\def\rarefactionDepth{')
        target.write(rarefactionDepth)
        target.write('}\n')
        
    # EMPO_3 PIE CHART OF PRESENCE/ABSENCE
    # value counts of empo_3 categories of samples OTU is found in
    empo3_count = df_map.loc[row['list_samples'].split(',')]['empo_3'].value_counts()
    # concat colors with counts and then remove zero values
    df_empo3 = pd.concat([df_colors, empo3_count], axis=1)
    df_empo3.columns = ['color', 'count']
    df_empo3_nonzero = df_empo3[df_empo3['count'] > 0]
    # draw pie chart
    fig = plt.figure(figsize=(4,4))
    fig.set_size_inches(5.6, 3)
    patches, text = plt.pie(df_empo3_nonzero['count'], labels=df_empo3_nonzero.index, colors=df_empo3_nonzero['color'], startangle=0)
    plt.axis('equal')
    plt.tight_layout()
    plt.savefig('%s/card_%sbp_subset%s_rare%s_rank%s_%s/pie.pdf' % (path_output, 
            trimLength, subset, rarefactionDepth, prevalenceRank, title))
    
    # EMPO_3 POINT PLOT OF PRESENCE/ABSENCE
    # value counts of empo3 counts in subset of samples
    vc_empo3_subset = df_map[df_map['observations_deblur_100bp'] >= 5000]['empo_3'].value_counts()
    df_empo3_nonzero.loc[:, 'count_all'] = vc_empo3_subset[df_empo3_nonzero.index] * 0.5
    # normalize counts to total
    df_empo3_nonzero['count_all_norm'] = df_empo3_nonzero['count_all'] / df_empo3_nonzero['count_all'].sum()
    df_empo3_nonzero['count_norm'] = df_empo3_nonzero['count'] / df_empo3_nonzero['count'].sum()
    # add empo column and reorder columns
    df_empo3_nonzero['empo'] = df_empo3_nonzero.index
    df_empo3_nonzero = df_empo3_nonzero[['empo', 'color', 'count_all', 'count', 'count_all_norm', 'count_norm']]
    # melt to format that is plot-able
    df_empo3_nonzero_melted = pd.melt(df_empo3_nonzero, id_vars=['empo', 'color'], value_vars=['count_norm', 'count_all_norm'])
    df_empo3_nonzero_melted.sort_values('variable', ascending=True, inplace=True)
    # point plot
    fig, ax = plt.subplots(figsize=(5,4))
    sns.pointplot(x='variable', y='value', hue='empo', data=df_empo3_nonzero_melted, palette=df_empo3_nonzero_melted['color'])
    plt.legend().set_visible(False)
    for empo in df_empo3_nonzero.index:
        mysize = 8+40*df_empo3_nonzero.loc[empo,'count_norm']
        if mysize > 16:
            mysize = 16
        plt.text(1.08, df_empo3_nonzero.loc[empo,'count_norm'], df_empo3_nonzero.loc[empo,'empo'], fontsize=mysize, va='center')
    plt.axis([-0.1, 2.5, -0.01, df_empo3_nonzero['count_norm'].max()*1.1])
    plt.box('off')
    plt.xticks([0, 1], ('All samples', 'Samples where OTU is found'))
    ax.tick_params(labelsize=10)
    plt.xlabel('')
    plt.ylabel('Relative distribution by EMPO sample type', fontsize=10)
    sns.despine(offset=10, trim=True)
    plt.tight_layout()
    plt.savefig('%s/card_%sbp_subset%s_rare%s_rank%s_%s/point.pdf' % (path_output, 
        trimLength, subset, rarefactionDepth, prevalenceRank, title))
    
    # ENVIRONMENTAL PARAMETER DISTRIBUTION PLOTS OF PRESENCE/ABSENCE
    # get ph values of samples OTU is found in
    ph_values = df_map.loc[row['list_samples'].split(',')]['ph']
    ph_values.dropna(inplace=True)
    all_ph_values = df_map[df_map[obs_column] >= 5000]['ph']
    all_ph_values.dropna(inplace=True)
    # get temperature values of samples OTU is found in
    temp_values = df_map.loc[row['list_samples'].split(',')]['temperature_deg_c']
    temp_values.dropna(inplace=True)
    all_temp_values = df_map[df_map[obs_column] >= 5000]['temperature_deg_c']
    all_temp_values.dropna(inplace=True)
    # get salinity values of samples OTU is found in
    sal_values = df_map.loc[row['list_samples'].split(',')]['salinity_psu']
    sal_values.dropna(inplace=True)
    all_sal_values = df_map[df_map[obs_column] >= 5000]['salinity_psu']
    all_sal_values.dropna(inplace=True)
    # draw dist plots
    fig, axes = plt.subplots(1, 3, figsize=(7, 2))
    sns.despine(left=True)
    # ph
    sns.distplot(all_ph_values, hist=False, rug=False, color='0.8', ax=axes[0], kde_kws={"shade": True})
    if ph_values.shape[0] > 1:
        sns.distplot(ph_values, hist=False, rug=True, color='g', ax=axes[0], axlabel='pH', kde_kws={"shade": True})
    # temp
    sns.distplot(all_temp_values, hist=False, rug=False, color='0.8', ax=axes[1], kde_kws={"shade": True})
    if temp_values.shape[0] > 1:
        sns.distplot(temp_values, hist=False, rug=True, color='r', ax=axes[1], axlabel='Temperature (°C)', kde_kws={"shade": True})
    # sal
    sns.distplot(all_sal_values, hist=False, rug=False, color='0.8', ax=axes[2], kde_kws={"shade": True})
    if sal_values.shape[0] > 1:
        sns.distplot(sal_values, hist=False, rug=True, color='b', ax=axes[2], axlabel='Salinity (psu)', kde_kws={"shade": True})
    axes[0].set_xlim([math.floor(all_ph_values.min()), math.ceil(all_ph_values.max())])
    axes[1].set_xlim([math.floor(all_temp_values.min()), math.ceil(all_temp_values.max())])
    axes[2].set_xlim([math.floor(all_sal_values.min()), math.ceil(all_sal_values.max())])
    plt.setp(axes, yticks=[])
    plt.tight_layout()
    plt.savefig('%s/card_%sbp_subset%s_rare%s_rank%s_%s/envparams.pdf' % (path_output, 
                trimLength, subset, rarefactionDepth, prevalenceRank, title))